# Parsing Logs from a MUD for Description Information

In [1]:
import os, sys
import re
import json
from triggers import *
from pathlib import Path

## 1.) Collect and load Logs from text source

In [2]:
data_folder = Path("data/mudlogs/")
idx = 0
doc = {}
document = []
for file in data_folder.iterdir():
    if str(file).endswith("DVDlog.log"):
        file_to_open = file
        o = open(file_to_open, 'r')
        documentName = idx
        idx += 1
        document = list(o)
        doc[documentName] = document
#print(docs[0])

In [ ]:
#SKIP FOR NOW
for line in document:
    ansi_rem = re.compile(r'\x1B\[[0-?]*[ -/]*[@-~](\\?\;?\d+)*m')
    
    #print(ansi_rem.sub('', line))

## 2.) Filter for descriptions (State)

In [5]:
description = []
index = -1
indices = []
descripDict = {}
for line in document: #in the end, append all good lines to a list then dict
    ansi_rem = re.split(r'(37m)|(33m)', line, 100)
    stripFront = ansi_rem
    l = len(ansi_rem)
    #print(stripFront[l-1])
    ansi_rem2 = re.split(r'(\\)', stripFront[l-1], 100)
    stripBack = ansi_rem2
    a = stripBack[0]
    l = len(a)
    if (a.startswith('{"mud": "sloth", "line": "') ):
        a = a[26:(l-74)]
    if (a.startswith('", "tag": "mudline",') ):
        a = ''
    if (a.startswith('",') ):
        a = ''
    if (a.startswith('"}') ):
        a = ''
    if (a.startswith('***') ):
        a = '' 
    if ('timestamp' in a):
        a = ''
    if ('http' in a):
        a = ''
    if (a != ''):
        #print(a) #for nice output
        description.append(a)
    index += 1
    indices.append(index)
descripDict = dict(zip(indices, description))

print(index) #for number of lines

84389


# Parsing SlothMUD Command List

In [ ]:
A
Alias
Apply
Ask
Assist
Attach
Setting:Autogroup
Setting:Autokill
Setting:Autoloot
Autoreport
Setting:Autosplit
B
Balance
Belch
Setting:Brief
Bugs
Buy
C
Carve
Cashin
Cast
Channels
Check
Claim
Close
Setting:Color
Commands
Setting:Compact
Compare
Setting:Concise
Consider
Convenience
CTalk
Setting:C Reset
D
Deposit
Description
Detach
Dismiss
Dismount
Drink
Drop
Eat
E
Setting:Echo
Emote
Enlist
Enter
Equipment
Examine
Exchange
Exits
F
Fill
Finger
Fire
Fishing
Flee
Follow
G
Get
Give
Gjob
Glock
Gmove
Gname
Gossip Channel
Grab
Group
Grouptell
Gset
Gunlock
Gwho
H
Help
Setting:Hint
Hold
Hunt
I
Inventory
Setting:Isnewbie
J
Junk
K
Kill
KTalk
L
Last Player
Leave
Setting:Lines
List
Listen
Load
Lock
Look
Lsplit
Lurk
M
Maneuver
Mount
Setting:MXP
Setting:MXPItems
N
Names
Nohunt
Notell
Setting:Numformat
O
Offer
Oops
Open
Order
P
Page
Play
Point
Position
Post
Pour
Pray
Private
Setting:Prompt
Settings:Protocol
Push
Put
Q
Quaff
Quit!
R
Read
Receive
Reclaim
Refollow
Release
Remove
Rent
Replay
Report
Respond
S
Sacrifice
Save
Say
Scan
Score
Score 2
Score Affects
Score Stats
Sell
Send
Settings
Ship
Shout
Setting:Showexits
Sing
Skills
Sleep
Socket
Spells
Split
Subdue
T
Teaching
Tell
Time
Time Started
Title
Setting:Toggle
U
Ungroup
Unlock
V
Value
Vomit
W
Wanted
Watch
Wear
Where
Whisper
Who
Wield
Withdraw
Write
Y
Yell

In [ ]:
import collections
import re

Token = collections.namedtuple('Token', ['type', 'value', 'line', 'column'])

def tokenize(code):
    keywords = {'IF', 'THEN', 'ENDIF', 'FOR', 'NEXT', 'GOSUB', 'RETURN'}
    token_specification = [
        ('NUMBER',   r'\d+(\.\d*)?'),  # Integer or decimal number
        ('ASSIGN',   r':='),           # Assignment operator
        ('END',      r';'),            # Statement terminator
        ('ID',       r'[A-Za-z]+'),    # Identifiers
        ('OP',       r'[+\-*/]'),      # Arithmetic operators
        ('NEWLINE',  r'\n'),           # Line endings
        ('SKIP',     r'[ \t]+'),       # Skip over spaces and tabs
        ('MISMATCH', r'.'),            # Any other character
    ]
    tok_regex = '|'.join('(?P<%s>%s)' % pair for pair in token_specification)
    line_num = 1
    line_start = 0
    for mo in re.finditer(tok_regex, code):
        kind = mo.lastgroup
        value = mo.group()
        column = mo.start() - line_start
        if kind == 'NUMBER':
            value = float(value) if '.' in value else int(value)
        elif kind == 'ID' and value in keywords:
            kind = value
        elif kind == 'NEWLINE':
            line_start = mo.end()
            line_num += 1
            continue
        elif kind == 'SKIP':
            continue
        elif kind == 'MISMATCH':
            raise RuntimeError(f'{value!r} unexpected on line {line_num}')
        yield Token(kind, value, line_num, column)

statements = '''{"mud": "sloth", "line": "\u001b[0\\;37mYou recall at least one encounter.  Counting beyond that is hard with all this equipment on your toes.\u001b[0\\;37\\;40m", "tag": "mudline", "user": "final", "timestamp": 1529199267.401862}'''

for token in tokenize(statements):
    print(token)

## 3.) Keras Encoder or LSTM-DQN from "Playing..."

## 4.) Sentiment Analysis (optional)

## 5.) Decoder

## 6.) Autoencoder with two outputs: reconstructed input and predicted sentiment